In [14]:
# 03_recommend_by_cosine_similarity.py

import pandas as pd
from janome.tokenizer import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# --- 準備：前処理用の関数 ---
def get_meaningful_tokens(text, tokenizer, stop_words):
    """文章を単語に分解し、重要な単語リストを返す"""
    tokens = []
    for token in tokenizer.tokenize(text):
        part_of_speech = token.part_of_speech.split(',')[0]
        if part_of_speech in ['名詞', '動詞', '形容詞'] and token.surface not in stop_words:
            tokens.append(token.base_form)
    return tokens

def text_to_bigram_string(text, tokenizer, stop_words):
    """テキストをBi-gramの文字列に変換する"""
    tokens = get_meaningful_tokens(text, tokenizer, stop_words)
    bigrams = [f"{tokens[i]}_{tokens[i+1]}" for i in range(len(tokens) - 1)]
    return ' '.join(bigrams)

# --- メイン処理 ---
if __name__ == '__main__':
    # 前処理済みCSVと分析ツールを準備
    try:
        df = pd.read_csv('csv/preprocessed_bigrams.csv')
    except FileNotFoundError:
        print("エラー: 'preprocessed_bigrams.csv' が見つかりません。")
        exit()

    df.dropna(subset=['bigram_corpus'], inplace=True)
    t = Tokenizer()
    stop_words = ['こと', 'もの', 'それ', 'これ', 'さん', 'よう', 'ため', 'こちら',
                   'ところ', '感じ', '利用', '訪問', '注文', '提供', '料理', 'お店', 
                   '雰囲気', '時間', '今回', '非常', '思う', 'いる', 'する', 'なる', 
                   'ある', 'ない', 'くる', 'いく', '良い'
                   ]
    
    # お店ごとの特徴ベクトル（TF-IDF）を作成
    corpus = df['bigram_corpus'].tolist()
    shop_names = df['shop_name'].tolist()
    
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(corpus)

    # 推薦ロジック
    
    # ユーザーの曖昧な要望を定義
    user_query = "パンケーキが美味しいカフェ"
    
    print(f"ユーザーの要望: 「{user_query}」\n")

    # ユーザーの要望を、お店と同じ形式のベクトルに変換
    query_bigrams = text_to_bigram_string(user_query, t, stop_words)
    query_vector = vectorizer.transform([query_bigrams])

    # コサイン類似度を計算
    #    ユーザーのベクトルと、全店舗のベクトル行列との類似度を一度に計算
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix)

    # 結果を整形してランキング表示
    #    計算結果から、各店舗のスコアをリストとして取り出す
    scores = cosine_similarities[0]
    
    # 店舗名とスコアをまとめたDataFrameを作成
    results_df = pd.DataFrame({
        'shop_name': shop_names,
        'score': scores
    })
    
    # スコアが高い順に並べ替え、上位10件を取得
    top_10_recommendations = results_df.sort_values(by='score', ascending=False).head(10)

    print("--- 推薦結果 TOP 10 ---")
    print(top_10_recommendations.to_string(index=False))

ユーザーの要望: 「パンケーキが美味しいカフェ」

--- 推薦結果 TOP 10 ---
                              shop_name    score
                              bills お台場 0.093936
                         エッグスンシングス お台場店 0.070992
         ARIAKE ARENA DINING MOON RIVER 0.058172
                  オリジナルパンケーキハウス 有明ガーデン店 0.034537
                   Seaside Café SOLARIS 0.024265
                        ビオラルカフェ 有明ガーデン店 0.023353
オールデイダイニング グランドエール(ヴィラフォンテーヌ グランド 東京有明) 0.012369
                       ラ・オハナ ダイバーシティ東京店 0.011690
                 ラムと豪州ジビエバル プラスアルファキッチン 0.000000
                            ワフーズ　タコス&モア 0.000000
